In [21]:
import numpy as np
import pandas as pd

In [22]:
brand_data = pd.read_csv("brand_dataset.csv")
influencer_data = pd.read_csv("influencer_dataset.csv")

In [23]:
brand_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location
0,1,Brand72,Nutrition Products,15-24,Female,Kerala
1,2,Brand88,Smartphones,All,Male,Gujarat
2,3,Brand55,Home Appliances,18-30,Male,Kerala
3,4,Brand84,Gadgets,15-24,Male,Kerala
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu


In [24]:
brand_data['brand_name'].unique

<bound method Series.unique of 0      Brand72
1      Brand88
2      Brand55
3      Brand84
4      Brand43
        ...   
595     Brand8
596    Brand80
597    Brand29
598     Brand7
599    Brand89
Name: brand_name, Length: 600, dtype: object>

In [25]:
influencer_data.head()

,id,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


In [26]:
brand_data.isnull().sum()

id                  0
brand_name          0
product_category    0
target_age          0
target_gender       0
target_location     0
dtype: int64

In [27]:

influencer_data.isnull().sum()

id                   0
Influencer_name      0
Handle               0
Followers            0
Posts                0
Likes                0
Category             0
Sentiment            0
audience_location    0
audience_age         0
audience_Gender      0
Platform             0
Engagement_rate      0
dtype: int64

merging both tables

In [28]:
merged_data = pd.merge(brand_data, influencer_data, on = 'id', how = 'right')

In [29]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749


Based on Search (using Category) --Using Stemming tokens

In [30]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stem = [stemmer.stem(w) for w in tokens]
    return " ".join(stem)


In [32]:
merged_data['stemmed_tokens'] =merged_data.apply(lambda row: tokenize_stem(row['Category']+ ' ' + row['audience_location']+ ' ' + row['audience_age']+ ' ' + row['Platform']), axis=1)

In [33]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate,stemmed_tokens
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,15-24,Male,TikTok,9.269302,food kerala 15-24 tiktok
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,All,All,Twitter,7.077254,fashion gujarat all twitter
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,18-30,Male,Instagram,36.526283,food kerala 18-30 instagram
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,15-24,All,Facebook,3.978293,lifestyl kerala 15-24 facebook
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,40-50,Female,Instagram,6.165749,lifestyl tamil nadu 40-50 instagram


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(tokenizer=tokenize_stem)

def cosine_sim(txt1,txt2):
    tfid_matrix = tfidvectorizer.fit_transform([txt1,txt2])
    return cosine_similarity(tfid_matrix)[0][1]

In [35]:
def search_product(query):
    stemmed_query = tokenize_stem(query)
    #calcualting cosine similarity between query and stemmed tokens columns
    merged_data['similarity'] = merged_data['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res = merged_data.sort_values(by=['similarity'],ascending=False).head(10)[['Influencer_name','Category','Handle','Followers','audience_location','audience_age','audience_Gender','Platform','Engagement_rate']]
    return res

In [36]:
# search based on category, audience_location, age, gender

search_product('Gaming Gujrat') 

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Influencer_name,Category,Handle,Followers,audience_location,audience_age,audience_Gender,Platform,Engagement_rate
292,Emma Taylor,Gaming,@olivia.c,448459,Gujarat,30-45,Male,Twitter,2.461986
99,Isabella Moore,Travel,@isabella.m,996589,Gujarat,25-40,Female,Instagram,3.665704
370,Emma Taylor,Travel,@olivia.c,942599,Gujarat,15-24,Female,Instagram,2.150437
437,Emma Taylor,Travel,@mason.w,621645,Gujarat,25-40,Female,Instagram,4.213981
308,Olivia Clark,Health,@john.smith,724486,Gujarat,25-34,Female,Instagram,6.887366
469,Olivia Clark,Gaming,@sophia.w,641938,Gujarat,40-50,All,YouTube,6.172714
331,Isabella Moore,Gaming,@john.smith,969595,Gujarat,22-40,Male,YouTube,0.587462
542,Mason White,Gaming,@mia.lee,707184,Punjab,18-30,Female,Instagram,1.861185
161,Mia Lee,Gaming,@liam.brown,761467,Rajasthan,18-25,All,Instagram,2.105541
597,Emma Taylor,Gaming,@john.smith,822197,Goa,18-25,Male,Instagram,5.059128


based on serach using --tags

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import os
from scipy.sparse import coo_matrix

In [38]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def clean_and_extract_tags(text):
    doc = nlp(text.lower())
    tags = [token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
    return ', '.join(tags)

columns_to_extract_tags_from = ['Category', 'audience_location', 'audience_age',	'audience_Gender',	'Platform']

for column in columns_to_extract_tags_from:
    merged_data[column] = merged_data[column].apply(clean_and_extract_tags)

In [39]:
# Concatenate the cleaned tags from all relevant columns
merged_data['Tags'] = merged_data[columns_to_extract_tags_from].apply(lambda row: ', '.join(row), axis=1)

In [40]:
merged_data.head()

,id,brand_name,product_category,target_age,target_gender,target_location,Influencer_name,Handle,Followers,Posts,...,Category,Sentiment,audience_location,audience_age,audience_Gender,Platform,Engagement_rate,stemmed_tokens,similarity,Tags
0,1,Brand72,Nutrition Products,15-24,Female,Kerala,Jane Doe,@emma.t,340770,986,...,food,Positive,kerala,"15, 24",male,tiktok,9.269302,food kerala 15-24 tiktok,0.276835,"food, kerala, 15, 24, male, tiktok"
1,2,Brand88,Smartphones,All,Male,Gujarat,Mia Lee,@jane.doe,702603,847,...,fashion,Positive,gujarat,,,twitter,7.077254,fashion gujarat all twitter,0.628678,"fashion, gujarat, , , twitter"
2,3,Brand55,Home Appliances,18-30,Male,Kerala,John Smith,@noah.d,113855,241,...,food,Positive,kerala,"18, 30",male,instagram,36.526283,food kerala 18-30 instagram,0.543933,"food, kerala, 18, 30, male, instagram"
3,4,Brand84,Gadgets,15-24,Male,Kerala,Emma Taylor,@jane.doe,202147,944,...,lifestyle,Negative,kerala,"15, 24",,facebook,3.978293,lifestyl kerala 15-24 facebook,0.315457,"lifestyle, kerala, 15, 24, , facebook"
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu,Mason White,@john.smith,495171,508,...,lifestyle,Negative,"tamil, nadu","40, 50",female,instagram,6.165749,lifestyl tamil nadu 40-50 instagram,0.502232,"lifestyle, tamil, nadu, 40, 50, female, instagram"


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_content = tfidf_vectorizer.fit_transform(merged_data['Tags'])
cosine_similarities_content = cosine_similarity(tfidf_matrix_content,tfidf_matrix_content)

In [66]:
def search_similar_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content, top_n=10):
    # Vectorize the user input
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    # Compute cosine similarity between user input and the existing items
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix_content).flatten()

    # Get the indices of the top N most similar items
    similar_item_indices = cosine_similarities.argsort()[-top_n:][::-1]

    # Extract the most similar items' details
    similar_items = merged_data.iloc[similar_item_indices][['Influencer_name', 'Category', 'Followers', 'audience_location', 'audience_age', 'audience_Gender', 'Platform', 'Engagement_rate']]
    
    
    
    # Print the results in table format
    print(similar_items.to_string(index=False))  # Display the table without index numbers

    return similar_items

# Example usage:
user_input = "food, gujarat"  # User provides tags
similar_items = search_similar_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content, top_n=5)


Influencer_name  Category  Followers audience_location audience_age audience_Gender Platform  Engagement_rate
    Emma Taylor      food     788120           gujarat                               youtube         4.967771
        Mia Lee      food     456819           gujarat       40, 50          female  youtube         2.770025
        Mia Lee   fashion     702603           gujarat                               twitter         7.077254
    Emma Taylor lifestyle     926247           gujarat                               twitter         4.663551
 Isabella Moore    health     314518           gujarat                       female facebook        11.447675


using argmax

In [67]:
def search_exact_match_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content):
    # Vectorize the user input
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    # Compute cosine similarity between user input and the existing items
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix_content).flatten()

    
    # Get the index of the highest matching item (exact or closest)
    best_match_index = cosine_similarities.argmax()

    # Return the most similar item from the original DataFrame
    return merged_data.iloc[[best_match_index]][['Influencer_name', 'Category', 'Followers', 'audience_location', 'audience_age', 'audience_Gender', 'Platform', 'Engagement_rate']]

# Example usage:
user_input = "food, gujarat"  # User provides tags
matched_item = search_exact_match_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content)

# Print the matched item in table format
print(matched_item.to_string(index=False))


TypeError: argpartition() missing required argument 'kth' (pos 0)

using thresold

In [120]:
def search_exact_match_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content, threshold=0.5):
    # Vectorize the user input
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    # Compute cosine similarity between user input and the existing items
    cosine_similarities = cosine_similarity(user_input_tfidf, tfidf_matrix_content).flatten()

    # Get all indices where the cosine similarity is above a threshold
    matching_indices = [i for i, similarity in enumerate(cosine_similarities) if similarity >= threshold]

    # If no matches found above threshold, return a message
    if not matching_indices:
        print(f"No matches found for '{user_input}' with a similarity above {threshold}.")
        return pd.DataFrame()

    # Return all the most similar items that meet the threshold
    return merged_data.iloc[matching_indices][['Influencer_name', 'Category', 'Followers', 'audience_location', 'audience_age', 'audience_Gender', 'Platform', 'Engagement_rate']]

# Example usage:
user_input = "food"  # User provides tags
matched_items = search_exact_match_items(user_input, merged_data, tfidf_vectorizer, tfidf_matrix_content, threshold=0.5)

# Print the matched items in table format
if not matched_items.empty:
    print(matched_items.to_string(index=False))


Influencer_name Category  Followers audience_location audience_age audience_Gender  Platform  Engagement_rate
   Olivia Clark     food     114466            kerala                              instagram        13.659951
     John Smith     food     303539           haryana                                youtube         6.885771
   Olivia Clark     food     985901           haryana                                twitter         3.942789
    Emma Taylor     food     788120           gujarat                                youtube         4.967771
     John Smith     food     811862           haryana                       female    tiktok         6.077634
 Isabella Moore     food     420775            kerala                              instagram         9.458262
    Mason White     food     343755         karnataka                         male    tiktok         2.629780
  Sophia Wilson     food     596300             delhi                                youtube         3.746604


using bm-25 to find similarity

In [96]:
!pip install rank_bm25


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
from rank_bm25 import BM25Okapi

# Tokenize the tags for BM25
tokenized_corpus = [doc.split() for doc in merged_data['Tags']]

# Initialize BM25 model
bm25 = BM25Okapi(tokenized_corpus)

# Query the model with user input
tokenized_query = user_input.split()
bm25_scores = bm25.get_scores(tokenized_query)


In [123]:
import pandas as pd
from rank_bm25 import BM25Okapi


# Tokenize the tags for BM25
tokenized_corpus = [tags.split(', ') for tags in merged_data['Tags']]

# Initialize BM25 model
bm25 = BM25Okapi(tokenized_corpus)

def search_similar_items(user_input, merged_data, bm25_model, top_n=5):
    # Tokenize user input
    tokenized_query = user_input.split(', ')

    # Get BM25 scores for the user input
    bm25_scores = bm25_model.get_scores(tokenized_query)

    # Get the indices of the top N items
    top_indices = bm25_scores.argsort()[-top_n:][::-1]

    # Return the most similar items from the original DataFrame
    return merged_data.iloc[top_indices][['Influencer_name', 'Category', 'Followers', 'audience_location', 'audience_age', 'audience_Gender', 'Platform', 'Engagement_rate']]

# Example usage
user_input = "food, kerala, 15, 24, tiktok"
similar_items = search_similar_items(user_input, merged_data, bm25, top_n=2)
print(similar_items)


    Influencer_name Category  Followers audience_location audience_age  \
0          Jane Doe     food     340770            kerala       15, 24   
214    Olivia Clark     food     868842             bihar       15, 24   

    audience_Gender Platform  Engagement_rate  
0              male   tiktok         9.269302  
214            male   tiktok         4.347856  
